In [1]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import json
import gdown
import sys
from tqdm import tqdm
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

In [2]:
df1 = pd.read_csv('realestateinvestar1.csv')
df2 = pd.read_csv('realestateinvestar2.csv')
# df3 = pd.read_csv('realestateinvestar3.csv')
# df4 = pd.read_csv('/content/realestateinvestor/realestateinvestar4.csv')
# df5 = pd.read_csv('/content/realestateinvestor/realestateinvestar5.csv')
# df6 = pd.read_csv('/content/realestateinvestor/realestateinvestar6.csv')

In [3]:
# suburbdf = pd.read_csv('/content/realestateinvestment.csv')

In [4]:
suburbdf = pd.DataFrame()
suburbdf = pd.concat([suburbdf,df1],ignore_index =True)
suburbdf = pd.concat([suburbdf,df2],ignore_index =True)
# suburbdf = pd.concat([suburbdf,df3],ignore_index =True)
# suburbdf = pd.concat([suburbdf,df4],ignore_index =True)
# suburbdf = pd.concat([suburbdf,df5],ignore_index =True)
suburbdf

,string_from_domain,string_from_realestateinvestor,Current vacancy rate,Rental stock available,Population,Rental population,Median listing price_House,Median listing price_Townhouses,Median listing price_Units,Median price change - last quarter_House,...,Median rent change - 1 year_Townhouses,Median rent change - 1 year_Units,Previous month sales_House,Previous month sales_Units,Stock on market previous month_House,Stock on market previous month_Units,Stock variance vs. last year_House,Stock variance vs. last year_Units,Average days on market_House,Average days on market_Units
0,oaks-tas-7303,"Investment Property Oaks, TAS, 7303",NaN,0,35.0,62.50%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gibraltar-range-nsw-2370,"Investment Property Gibraltar Range, NSW, 2370",NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bulli-creek-qld-4357,"Investment Property Bulli Creek, QLD, 4357",NaN,0,8.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,gurindji-nt-0852,"Investment Property Gurindji, NT, 0852",NaN,0,21.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,buckenderra-nsw-2630,"Investment Property Buckenderra, NSW, 2630",NaN,0,18.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15075,bindi-vic-3896,"Investment Property Bindi, VIC, 3896",NaN,0,56,13.64%,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.00%,0.00%,NaN,NaN
15076,mongarlowe-nsw-2622,"Investment Property Mongarlowe, NSW, 2622",NaN,0,118,3.75%,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,1.0,0.0,100.00%,0.00%,NaN,NaN
15077,teal-point-vic-3579,"Investment Property Teal Point, VIC, 3579",NaN,0,67,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15078,macdonalds-creek-nsw-2339,"Investment Property Macdonalds Creek, NSW, 2339",NaN,0,12,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = suburbdf

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df['string_from_realestateinvestor'] = df['string_from_realestateinvestor'].apply(lambda x: str(x).strip('Investment Property'))

In [8]:
new_column_names = {col: col.lower().replace(' ', '_') for col in df.columns}
df = df.rename(columns = new_column_names)
df

,string_from_domain,string_from_realestateinvestor,current_vacancy_rate,rental_stock_available,population,rental_population,median_listing_price_house,median_listing_price_townhouses,median_listing_price_units,median_price_change_-_last_quarter_house,...,median_rent_change_-_1_year_townhouses,median_rent_change_-_1_year_units,previous_month_sales_house,previous_month_sales_units,stock_on_market_previous_month_house,stock_on_market_previous_month_units,stock_variance_vs._last_year_house,stock_variance_vs._last_year_units,average_days_on_market_house,average_days_on_market_units
0,oaks-tas-7303,"Oaks, TAS, 7303",NaN,0,35.0,62.50%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gibraltar-range-nsw-2370,"Gibraltar Range, NSW, 2370",NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bulli-creek-qld-4357,"Bulli Creek, QLD, 4357",NaN,0,8.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,gurindji-nt-0852,"Gurindji, NT, 0852",NaN,0,21.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,buckenderra-nsw-2630,"Buckenderra, NSW, 2630",NaN,0,18.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15075,bindi-vic-3896,"Bindi, VIC, 3896",NaN,0,56,13.64%,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.00%,0.00%,NaN,NaN
15076,mongarlowe-nsw-2622,"Mongarlowe, NSW, 2622",NaN,0,118,3.75%,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,1.0,0.0,100.00%,0.00%,NaN,NaN
15077,teal-point-vic-3579,"Teal Point, VIC, 3579",NaN,0,67,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15078,macdonalds-creek-nsw-2339,"Macdonalds Creek, NSW, 2339",NaN,0,12,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df['string_from_realestateinvestor'] = df['string_from_realestateinvestor'].apply(lambda x: '-'.join(str(x).split(',')).lower())
df['string_from_realestateinvestor'] = df['string_from_realestateinvestor'].str.replace(' ', '-')
df['string_from_realestateinvestor'] = df['string_from_realestateinvestor'].str.replace('--', '-')

In [10]:
df.columns

Index(['string_from_domain', 'string_from_realestateinvestor',
       'current_vacancy_rate', 'rental_stock_available', 'population',
       'rental_population', 'median_listing_price_house',
       'median_listing_price_townhouses', 'median_listing_price_units',
       'median_price_change_-_last_quarter_house',
       'median_price_change_-_last_quarter_townhouses',
       'median_price_change_-_last_quarter_units',
       'median_price_change_-_1_year_house',
       'median_price_change_-_1_year_townhouses',
       'median_price_change_-_1_year_units',
       'median_price_change_-_2_years_house',
       'median_price_change_-_2_years_townhouses',
       'median_price_change_-_2_years_units', 'median_weekly_rent_house',
       'median_weekly_rent_townhouses', 'median_weekly_rent_units',
       'median_yield_%_house', 'median_yield_%_townhouses',
       'median_yield_%_units', 'median_rent_change_-_1_year_house',
       'median_rent_change_-_1_year_townhouses',
       'median_rent_ch

In [11]:
df = df.drop_duplicates(subset=df.iloc[:,1:])
df.reset_index(drop=True, inplace = True)

In [12]:
df

,string_from_domain,string_from_realestateinvestor,current_vacancy_rate,rental_stock_available,population,rental_population,median_listing_price_house,median_listing_price_townhouses,median_listing_price_units,median_price_change_-_last_quarter_house,...,median_rent_change_-_1_year_townhouses,median_rent_change_-_1_year_units,previous_month_sales_house,previous_month_sales_units,stock_on_market_previous_month_house,stock_on_market_previous_month_units,stock_variance_vs._last_year_house,stock_variance_vs._last_year_units,average_days_on_market_house,average_days_on_market_units
0,oaks-tas-7303,oaks-tas-7303,NaN,0,35.0,62.50%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gibraltar-range-nsw-2370,gibraltar-range-nsw-2370,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bulli-creek-qld-4357,bulli-creek-qld-4357,NaN,0,8.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,gurindji-nt-0852,gurindji-nt-0852,NaN,0,21.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,buckenderra-nsw-2630,buckenderra-nsw-2630,NaN,0,18.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14338,bindi-vic-3896,bindi-vic-3896,NaN,0,56,13.64%,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.00%,0.00%,NaN,NaN
14339,mongarlowe-nsw-2622,mongarlowe-nsw-2622,NaN,0,118,3.75%,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,1.0,0.0,100.00%,0.00%,NaN,NaN
14340,teal-point-vic-3579,teal-point-vic-3579,NaN,0,67,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14341,macdonalds-creek-nsw-2339,macdonalds-creek-nsw-2339,NaN,0,12,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# gdown.download(id = '1DuT3nnIouLKuRFbGo5lqcQwKTTb5GM8r')
# sdf = pd.read_csv('suburbs.csv')

In [14]:
# merged_table = pd.merge(df, sdf['0'], how='outer', left_on='string_from_realestateinvestor',right_on='0' ,indicator=True)

In [15]:
# merged_table[merged_table['_merge']=='both'].iloc[:,[0,1,-1,-2]]

In [16]:
df

,string_from_domain,string_from_realestateinvestor,current_vacancy_rate,rental_stock_available,population,rental_population,median_listing_price_house,median_listing_price_townhouses,median_listing_price_units,median_price_change_-_last_quarter_house,...,median_rent_change_-_1_year_townhouses,median_rent_change_-_1_year_units,previous_month_sales_house,previous_month_sales_units,stock_on_market_previous_month_house,stock_on_market_previous_month_units,stock_variance_vs._last_year_house,stock_variance_vs._last_year_units,average_days_on_market_house,average_days_on_market_units
0,oaks-tas-7303,oaks-tas-7303,NaN,0,35.0,62.50%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gibraltar-range-nsw-2370,gibraltar-range-nsw-2370,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bulli-creek-qld-4357,bulli-creek-qld-4357,NaN,0,8.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,gurindji-nt-0852,gurindji-nt-0852,NaN,0,21.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,buckenderra-nsw-2630,buckenderra-nsw-2630,NaN,0,18.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14338,bindi-vic-3896,bindi-vic-3896,NaN,0,56,13.64%,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.00%,0.00%,NaN,NaN
14339,mongarlowe-nsw-2622,mongarlowe-nsw-2622,NaN,0,118,3.75%,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,1.0,0.0,100.00%,0.00%,NaN,NaN
14340,teal-point-vic-3579,teal-point-vic-3579,NaN,0,67,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14341,macdonalds-creek-nsw-2339,macdonalds-creek-nsw-2339,NaN,0,12,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df['string_from_realestateinvestor'] = df.apply(lambda row: row['string_from_domain'] if row['string_from_realestateinvestor']=='a' else row['string_from_realestateinvestor'], axis=1)

C:\Users\PM\AppData\Local\Temp\ipykernel_12072\3187065409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['string_from_realestateinvestor'] = df.apply(lambda row: row['string_from_domain'] if row['string_from_realestateinvestor']=='a' else row['string_from_realestateinvestor'], axis=1)


In [18]:
df['string_from_realestateinvestor'] = df.apply(lambda row: row['string_from_domain'] if row['string_from_realestateinvestor'].split('-')[-1] == row['string_from_domain'].split('-')[-1] else row['string_from_realestateinvestor'], axis=1)

C:\Users\PM\AppData\Local\Temp\ipykernel_12072\4137628750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['string_from_realestateinvestor'] = df.apply(lambda row: row['string_from_domain'] if row['string_from_realestateinvestor'].split('-')[-1] == row['string_from_domain'].split('-')[-1] else row['string_from_realestateinvestor'], axis=1)


In [19]:
df['postcode'] = df['string_from_realestateinvestor'].apply(lambda x: x[-4:].title())
df['state'] = df['string_from_realestateinvestor'].apply(lambda x: x.split('-')[-2].title())
df['suburb_Name'] = df['string_from_realestateinvestor'].apply(lambda x: ' '.join(x.split('-')[:-2]).title())
df['postcode'] = df['postcode'].apply(lambda x: str(x).zfill(4))

C:\Users\PM\AppData\Local\Temp\ipykernel_12072\1077596171.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['postcode'] = df['string_from_realestateinvestor'].apply(lambda x: x[-4:].title())
C:\Users\PM\AppData\Local\Temp\ipykernel_12072\1077596171.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['state'] = df['string_from_realestateinvestor'].apply(lambda x: x.split('-')[-2].title())
C:\Users\PM\AppData\Local\Temp\ipykernel_12072\1077596171.py:3: SettingWithCopyWarning: 
A value is trying to be 

In [20]:
state_names = {
    'Tas': 'Tasmania',
    'Nsw': 'New South Wales',
    'Qld': 'Queensland',
    'Nt': 'Northern Territory',
    'Vic': 'Victoria',
    'Sa': 'South Australia',
    'Act': 'Australian Capital Territory',
    'Wa': 'Western Australia'
    }
df['state'] = df['state'].replace(state_names)

C:\Users\PM\AppData\Local\Temp\ipykernel_12072\3236426793.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['state'] = df['state'].replace(state_names)


In [21]:
df['current_vacancy_rate'] = df['current_vacancy_rate'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
df['rental_stock_available'] = df['rental_stock_available'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace(',','')))
df['population'] = df['population'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace(',', '').replace('+', '')))
df['rental_population'] = df['rental_population'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))

C:\Users\PM\AppData\Local\Temp\ipykernel_12072\2835901194.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['current_vacancy_rate'] = df['current_vacancy_rate'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
C:\Users\PM\AppData\Local\Temp\ipykernel_12072\2835901194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rental_stock_available'] = df['rental_stock_available'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace(',','')))
C:\Users\PM\AppData\Local\Temp\ipy

In [22]:
df['median_listing_price_house'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace(',', '').replace('$', '')))

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
14338   NaN
14339   NaN
14340   NaN
14341   NaN
14342   NaN
Name: median_listing_price_house, Length: 14343, dtype: float64

In [23]:
columns_to_clean = ['median_listing_price_house','median_listing_price_townhouses', 'median_listing_price_units','median_weekly_rent_house','median_weekly_rent_townhouses','median_weekly_rent_units']
for column in columns_to_clean:
    df[column] = df[column].apply(lambda x: np.NaN if x == '-' else float(str(x).replace(',', '').replace('$', '')))

C:\Users\PM\AppData\Local\Temp\ipykernel_12072\1212720710.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(lambda x: np.NaN if x == '-' else float(str(x).replace(',', '').replace('$', '')))


In [24]:
df.columns

Index(['string_from_domain', 'string_from_realestateinvestor',
       'current_vacancy_rate', 'rental_stock_available', 'population',
       'rental_population', 'median_listing_price_house',
       'median_listing_price_townhouses', 'median_listing_price_units',
       'median_price_change_-_last_quarter_house',
       'median_price_change_-_last_quarter_townhouses',
       'median_price_change_-_last_quarter_units',
       'median_price_change_-_1_year_house',
       'median_price_change_-_1_year_townhouses',
       'median_price_change_-_1_year_units',
       'median_price_change_-_2_years_house',
       'median_price_change_-_2_years_townhouses',
       'median_price_change_-_2_years_units', 'median_weekly_rent_house',
       'median_weekly_rent_townhouses', 'median_weekly_rent_units',
       'median_yield_%_house', 'median_yield_%_townhouses',
       'median_yield_%_units', 'median_rent_change_-_1_year_house',
       'median_rent_change_-_1_year_townhouses',
       'median_rent_ch

In [25]:
columns_to_clean = ['median_price_change_-_last_quarter_house','median_price_change_-_last_quarter_townhouses','median_price_change_-_last_quarter_units','median_price_change_-_1_year_house',
 'median_price_change_-_1_year_townhouses','median_price_change_-_1_year_units','median_price_change_-_2_years_house','median_price_change_-_2_years_townhouses','median_price_change_-_2_years_units',
 	'median_yield_%_house',	'median_yield_%_townhouses','median_yield_%_units','median_rent_change_-_1_year_house','median_rent_change_-_1_year_townhouses','median_rent_change_-_1_year_units',
  'stock_variance_vs._last_year_house','stock_variance_vs._last_year_units']

# columns_to_clean = ['median_price_change___last_quarter_house',
#        'median_price_change___last_quarter_townhouses',
#        'median_price_change__last_quarter_units',
#        'median_price_change___1__year_house',
#        'median_price_change__1__year_townhouses',
#        'median_price_change__1_year_units',
#        'median_price_change_2_years_house',
#        'median_price_change_2_years_townhouses',
#        'median_price_change_2_years_units',
#        'median_yield_%_house', 'median_yield_%_townhouses',
#        'median_yield_%_units', 'median_rent_change_1_year_house',
#        'median_rent_change_1_year_townhouses',
#        'median_rent_change_1_year_units',
#  	     'stock_variance_vs_last_year_house',
#        'stock_variance_vs_last_year_units']

for column in columns_to_clean:
    df[column] = df[column].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))

C:\Users\PM\AppData\Local\Temp\ipykernel_12072\3601516680.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))


In [26]:
# df.iloc[:,23].unique()
df.iloc[:,31:33]

,stock_variance_vs._last_year_house,stock_variance_vs._last_year_units
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
14338,0.0,0.0
14339,100.0,0.0
14340,NaN,NaN
14341,NaN,NaN


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14343 entries, 0 to 14342
Data columns (total 38 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   string_from_domain                             14343 non-null  object 
 1   string_from_realestateinvestor                 14343 non-null  object 
 2   current_vacancy_rate                           5158 non-null   float64
 3   rental_stock_available                         14343 non-null  float64
 4   population                                     13147 non-null  float64
 5   rental_population                              10061 non-null  float64
 6   median_listing_price_house                     4024 non-null   float64
 7   median_listing_price_townhouses                981 non-null    float64
 8   median_listing_price_units                     1285 non-null   float64
 9   median_price_change_-_last_quarter_house       393

In [28]:
new_column_names = {col: re.sub(r'\W+', '_', col.lower().replace(' ', '_').replace('___', '_').replace('__', '_')) for col in df.columns}
df = df.rename(columns=new_column_names)

new_column_names = {col: re.sub(r'\W+', '_', col.lower().replace(' ', '_').replace('___', '_').replace('__', '_')) for col in df.columns}
df = df.rename(columns=new_column_names)
df.columns

Index(['string_from_domain', 'string_from_realestateinvestor',
       'current_vacancy_rate', 'rental_stock_available', 'population',
       'rental_population', 'median_listing_price_house',
       'median_listing_price_townhouses', 'median_listing_price_units',
       'median_price_change_last_quarter_house',
       'median_price_change_last_quarter_townhouses',
       'median_price_change_last_quarter_units',
       'median_price_change_1_year_house',
       'median_price_change_1_year_townhouses',
       'median_price_change_1_year_units', 'median_price_change_2_years_house',
       'median_price_change_2_years_townhouses',
       'median_price_change_2_years_units', 'median_weekly_rent_house',
       'median_weekly_rent_townhouses', 'median_weekly_rent_units',
       'median_yield_house', 'median_yield_townhouses', 'median_yield_units',
       'median_rent_change_1_year_house',
       'median_rent_change_1_year_townhouses',
       'median_rent_change_1_year_units', 'previous_month

In [29]:
custom = ['postcode', 'state', 'suburb_name','string_from_domain',
 'string_from_realestateinvestor', 'population', 'rental_population','current_vacancy_rate', 'rental_stock_available',
 'median_listing_price_house','median_listing_price_townhouses', 'median_listing_price_units',
       'median_price_change_last_quarter_house',
       'median_price_change_last_quarter_townhouses',
       'median_price_change_last_quarter_units',
       'median_price_change_1_year_house',
       'median_price_change_1_year_townhouses',
       'median_price_change_1_year_units',
       'median_price_change_2_years_house',
       'median_price_change_2_years_townhouses',
       'median_price_change_2_years_units',
       'median_weekly_rent_house','median_weekly_rent_townhouses', 'median_weekly_rent_units',
       'median_yield_house', 'median_yield_townhouses', 'median_yield_units',
       'median_rent_change_1_year_house',
       'median_rent_change_1_year_townhouses',
       'median_rent_change_1_year_units', 'previous_month_sales_house','previous_month_sales_units',
       'stock_on_market_previous_month_house','stock_on_market_previous_month_units',
       'stock_variance_vs_last_year_house','stock_variance_vs_last_year_units',
       'average_days_on_market_house','average_days_on_market_units']

df = df.reindex(columns = custom)

In [30]:
df = df.sort_values(by=['postcode', 'suburb_name'])
df = df.reset_index(drop=True)
df

,postcode,state,suburb_name,string_from_domain,string_from_realestateinvestor,population,rental_population,current_vacancy_rate,rental_stock_available,median_listing_price_house,...,median_rent_change_1_year_townhouses,median_rent_change_1_year_units,previous_month_sales_house,previous_month_sales_units,stock_on_market_previous_month_house,stock_on_market_previous_month_units,stock_variance_vs_last_year_house,stock_variance_vs_last_year_units,average_days_on_market_house,average_days_on_market_units
0,0800,Northern Territory,Darwin City,darwin-city-nt-0800,darwin-city-nt-0800,6464.0,50.18,4.10,177.0,NaN,...,NaN,2.72,0.0,34.0,3.0,223.0,-40.00,14.95,NaN,226.0
1,0810,Northern Territory,Alawa,alawa-nt-0810,alawa-nt-0810,2133.0,27.50,1.50,12.0,579000.0,...,NaN,11.62,1.0,0.0,12.0,4.0,0.00,-55.56,75.0,228.0
2,0810,Northern Territory,Brinkin,brinkin-nt-0810,brinkin-nt-0810,1274.0,35.00,1.26,5.0,NaN,...,NaN,-2.39,1.0,1.0,4.0,6.0,-20.00,-14.29,NaN,123.0
3,0810,Northern Territory,Coconut Grove,coconut-grove-nt-0810,coconut-grove-nt-0810,3057.0,45.85,1.42,22.0,NaN,...,NaN,13.63,5.0,4.0,11.0,21.0,-26.67,10.53,NaN,209.0
4,0810,Northern Territory,Jingili,jingili-nt-0810,jingili-nt-0810,1755.0,21.36,0.45,3.0,557500.0,...,NaN,NaN,1.0,0.0,12.0,0.0,-14.29,0.00,85.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14338,7468,Tasmania,Strahan,strahan-tas-7468,strahan-tas-7468,705.0,21.88,0.38,2.0,430000.0,...,NaN,NaN,0.0,0.0,18.0,0.0,125.00,-100.00,139.0,NaN
14339,7469,Tasmania,Granville Harbour,granville-harbour-tas-7469,granville-harbour-tas-7469,28.0,NaN,NaN,0.0,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.00,0.00,NaN,NaN
14340,7469,Tasmania,Renison Bell,renison-bell-tas-7469,renison-bell-tas-7469,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14341,7469,Tasmania,Trial Harbour,trial-harbour-tas-7469,trial-harbour-tas-7469,22.0,NaN,2.17,1.0,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.00,0.00,NaN,NaN


In [31]:
# filename = 'Realestateinvestor_Extract_20230708.xlsx'
# df.to_excel(filename,index = False)

In [32]:
australia_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.datetime.now(australia_timezone)
filename = 'Realestateinvestor_Extract_'+ str(current_time.date()).replace('-', '')+'.xlsx'
df.to_excel(filename,index = False)

In [33]:
df = pd.read_excel(filename) 
# df = pd.read_excel('Realestateinvestor_Extract_20231015.xlsx')

In [34]:
df.rename(columns={'string_from_domain':'domain_search_string'},inplace=True)

In [35]:
from sqlalchemy import create_engine
import mysql.connector

# Define your database connection parameters
host = "54.79.186.42"
user = "root"
password = "secondstorey"
database = "secondstorey"

# Establish a connection to the database
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

# Check if the connection was successful
if connection.is_connected():
    print("Connected to MySQL database")

# Close the connection when done
# Define the SQLAlchemy connection string
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Create the table in MySQL (if_exists='replace' will create the table if it doesn't exist)
df.to_sql(name='real_estate_investment', con=engine, if_exists='replace', index=False)
engine.dispose()

Connected to MySQL database


In [36]:
'https://onedrive.live.com/?id=F87FC3AA6EB46E0%21197311&cid=0F87FC3AA6EB46E0'

'https://onedrive.live.com/?id=F87FC3AA6EB46E0%21197311&cid=0F87FC3AA6EB46E0'